In [1]:
from bs4 import BeautifulSoup
import urllib.request as urllib
from scrapy.http import TextResponse
import requests

# r = urllib.urlopen('https://www.kerbfood.com/markets/west-india-quay/').read()
# soup = BeautifulSoup(r)

kerb_req = requests.get('https://www.kerbfood.com/markets/west-india-quay/')
response = TextResponse(kerb_req.url, body=kerb_req.text, encoding='utf-8')

In [2]:
# get dates from website
dates_xpath = response.xpath('//*[@class="rsCaption"]/text()')
dates = dates_xpath.getall()
dates


['Thursday 5th September',
 'Friday 6th September',
 'Monday 9th September',
 'Tuesday 10th September',
 'Wednesday 11th September',
 'Thursday 12th September',
 'Friday 13th September',
 'Monday 16th September',
 'Tuesday 17th September',
 'Wednesday 18th September',
 'Thursday 19th September',
 'Friday 20th September',
 'Monday 23rd September',
 'Tuesday 24th September',
 'Wednesday 25th September',
 'Thursday 26th September',
 'Friday 27th September',
 'Monday 30th September']

In [22]:
# parse dates
import dateutil
from datetime import date, timedelta, datetime

def parse_date(date_string):
    return dateutil.parser.parse(date_string).date()

def after_today(compare_date):
    return compare_date > datetime.today().date()

def before_today(compare_date):
    return compare_date < datetime.today().date()

def after_date(compare_date, after_date):
    return compare_date > after_date

def before_date(compare_date, before_date):
    return compare_date < before_date

def end_of_this_week():
    today = datetime.today().weekday()
    days_til_eow = 6 - today
    return datetime.today().date() + timedelta(days=days_til_eow)

def end_of_week(from_date):
#     today = date.today().weekday()
    day = from_date.weekday()
    days_til_eow = 6 - day # if day < 6 else day
    return from_date + timedelta(days=days_til_eow)
    
end_of_week(date(2019, 9, 8))

datetime.date(2019, 9, 8)

In [23]:
# from dates only take those between now and end of week

this_week_dates = [date for date in dates if after_today(parse_date(date)) and before_date(parse_date(date), end_of_this_week())]
this_week_dates

['Friday 6th September']

In [27]:
def get_day_traders(day):
    return response.xpath('//*[@class="rsCaption"][text()="{}"]/following-sibling::div[contains(@class, "row")]/div[contains(@class, "grid-item")]//span[contains(@class, "bolder")]/text()'.format(day)).getall()
# len(grid_items_xpath)

traders = {}
for day in this_week_dates:
    day_traders = get_day_traders(day)
    traders[day] = day_traders
    
traders

{'Friday 6th September': ['Baggio Burger',
  'Komex Kitchen',
  'Lovely Bunch of Coconuts',
  'Makatcha',
  'Mother Flipper',
  'Nazari',
  'The Beefsteaks',
  'The Grilling Greek',
  'The Patate',
  'Mexikings',
  'Hanoi Kitchen']}

In [47]:
import random 

kerb_days = 2 # CHANGE THIS NUMBER FOR HOW MANY DAYS YOU WANNA KERB


try:
    random_date_no = random.sample(range(1, len(this_week_dates)), kerb_days) if len(this_week_dates) > 1 else [len(this_week_dates)]
except:
    print("Kerb isn't on that many days.....")
    random_date_no = random.sample(range(1, len(this_week_dates)), len(this_week_dates))

for date_no in random_date_no:
    chosen_date = this_week_dates[date_no - 1]
    date_traders = traders[chosen_date]
    
    random_trader_no = random.randint(1, len(date_traders))
    print('Go to {}\n on {}'.format(date_traders[random_trader_no - 1], chosen_date))
    

Go to Mother Flipper
 on Friday 6th September


In [49]:
print('Kerb for the Week:')
kerb_days = 2
for date in dates:
    daily = response.xpath('//*[@class="rsCaption"][text()="{}"]/following-sibling::div[contains(@class, "row")]/div[contains(@class, "grid-item")]'.format(date))
    daily_traders = daily.xpath('.//span[contains(@class, "bolder")]/text()').getall()
    daily_random_trader = random.randint(1, len(daily_traders))
    print('{}: {}'.format(date, daily_traders[daily_random_trader-1]))
    
    

Kerb for the Week:
Thursday 5th September: Cally Munchy
Friday 6th September: Nazari
Monday 9th September: Ugly Duck
Tuesday 10th September: Yaay Yaay
Wednesday 11th September: Turo Turo
Thursday 12th September: Meltsmiths
Friday 13th September: Nazari
Monday 16th September: Ugly Duck
Tuesday 17th September: Lagniappe Trini Kitchen
Wednesday 18th September: Carcass
Thursday 19th September: Meltsmiths
Friday 20th September: Makatcha
Monday 23rd September: Utter Waffle
Tuesday 24th September: Tribeca Deli
Wednesday 25th September: Rice Guys
Thursday 26th September: Meltsmiths
Friday 27th September: The Patate
Monday 30th September: Ugly Duck
